# LORIS API Tour 2/2

## Setup

In [ ]:
import getpass  # For input prompt not to show what is entered
import json     # Provide convinent functions to handle json objects 
import re       # For regular expression
import requests # To handle http requests

import warnings
warnings.simplefilter('ignore') # Because I am using unverified ssl certificates 

baseurl = 'https://demo.loris.ca/api/v0.0.3-dev' # Pick yours

def prettyPrint(string):
    print(json.dumps(string, indent=2, sort_keys=True))

## Exercise 1. Login

This is a POST request to the `/login` endpoint that requires 2 parameters: `username` and `password`  
The expected response is a json string that contains a token property.  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#11-authentication

In [ ]:
payload = {
    'username': input('username: '), 
    'password': getpass.getpass('password: ')
}

response = requests.post(
    url = baseurl + '/login',
    json = payload,
    verify = False
)

text = response.content.decode('ascii')

data = json.loads(text)

prettyPrint(data)

*Store the token in a variable for later*

In [ ]:
token = data['token']

## Exercise 2. Get all candid

This is a GET request to /candidates  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#30-candidate-api

In [ ]:
response = json.loads(requests.get(
    url = baseurl + '/candidates',
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

In [ ]:
prettyPrint(response)

In [ ]:
# Using a for loop, print the candids
candidates = response['Candidates']
for candidate in candidates:
    print(candidate['CandID'])

## Exercise 3. Create a candidate
 - Send a **POST** request to /candidates with a payload containing an object with a candidate property
 ```json
    "Candidate" : {
        "Project" : ProjectName,
        "PSCID"   : PSCID, # only if config is set to prompt 
        "EDC"     : "YYYY-MM-DD", # if useEDC config is true (bug: always required)
        "DoB"     : "YYYY-MM-DD",
        "Gender"  : "Male|Female",
        "Site"    : SiteName,
    }
 ```

In [ ]:
# Find the project name and site name.
projects = json.loads(requests.get(
    url = baseurl + '/projects',
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

prettyPrint(projects)

In [ ]:
projectname = list(projects['Projects'])[0]

# TODO:: There is no way to know the existing sites of a LORIS instance with the API
sitename = 'Montreal'

json_data = {
    'Candidate' : {
        'Project' : projectname,
        'DoB'     : "2015-09-10",
        'EDC'     : "2015-09-10", #EDC seems to be always required even when useEDC is false...
        'Gender'  : "Female",
        'Site'    : sitename,
    }
}

r = requests.post(
    url = baseurl + '/candidates/',
    json = json_data,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r)
print(r.headers)
print(r.content.decode('ascii'))

In [ ]:
# Get the new candidate
candid = json.loads(r.content.decode('ascii'))['CandID']

newcandidate = json.loads(requests.get(
    url = baseurl + '/candidates/' + str(candid),
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

prettyPrint(newcandidate)

## Exercise 4. Add a timepoint to the new candidate
 - **PUT** request to /candidates/\$candid/\$visit_label
```json
{
    "Meta" : {
        "CandID" : CandID,
        "Visit"  : VisitLabel,
        "Site"   : SiteName,
        "Battery": "NameOfSubproject"
}
```

 - VisitLabel can be found using /projects/\$projectname (not realy; in fact, it is only every existing visit_labels...)
 - Battery (NameOfSubproject) must be guessed... 

In [ ]:

visitlabel = json.loads(requests.get(
    url = baseurl + '/projects/' + projectname,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))['Visits'][0]

print(visitlabel)

json_data = {
    'Meta' : {
        'CandID' : candid,
        'Visit'  : visitlabel,
        'Site'   : sitename,
        'Battery': "Control"
    }
}

r = requests.put(
    url = baseurl + '/candidates/' + str(candid) + '/' + visitlabel,
    json = json_data,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r)
print(r.content.decode('ascii'))

In [ ]:
r = requests.get(
    url = baseurl + '/candidates/' + str(candid) + '/' + visitlabel,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r)
prettyPrint(json.loads(r.content.decode('ascii')))

*There is no way to start the visit ... *

## Exercise 5. Input instrument data for a candidate
  
PUT or PATCH request to /candidates/$CandID/$VisitLabel/instruments/$InstrumentName  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "DDE": true|false,
    "Instrument": string,
    "Visit": string
  },
  <instrument_name>: {
    <field1_name>: <value1>,
    <field2_name>: <value2>,
    ...
  }
}
```


In [ ]:
def getInstrumentData(candid, visit_label, instrument):
    return json.loads(requests.get(
        url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))
    
instrument = 'test_all_fields'
candid = '279656'
visit_label = 'V01'

prettyPrint(getInstrumentData(candid, visit_label, instrument))

#### PUT request containing all the fields

In [ ]:
# Get all the fields an meta data
json_input = getInstrumentData(candid, visit_label, instrument)

# Update one field
json_input[instrument]['numeric_1'] = 3

r = requests.put(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = json_input,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r.status_code) # <Response [500]> demo.loris.ca do not handle that on LORIS 16...

prettyPrint(getInstrumentData(candid, visit_label, instrument))

#### PATCH request containing some of the fields

In [ ]:
# Get all the meta data
old_values = getInstrumentData(candid, visit_label, instrument)
new_values = {}
new_values['Meta'] = old_values['Meta']

# Add fields in the instrument object
new_values[instrument] = {}
new_values[instrument]['score_field_1'] = 42
new_values[instrument]['textbox_1'] = 'One Ring to rule them all'

prettyPrint(new_values)


In [ ]:
r = requests.patch(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = new_values,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r.status_code) # <Response [500]> demo.loris.ca do not handle that on LORIS 16...

prettyPrint(getInstrumentData(candid, visit_label, instrument))

## Exercise 6. Find new images

In [ ]:
# This gets all the images
r = json.loads(requests.get(
    url = baseurl + '/projects/loris/images' ,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))
prettyPrint(r)

In [ ]:
# This gets all the images inserted since 2018-12-13T10:20:18-05:00
r = json.loads(requests.get(
    url = baseurl + '/projects/loris/images?since=2018-12-13T10:20:18-05:00' ,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))
prettyPrint(r)

## Exercise 7. Download minc files

In [ ]:
candid = '279656'
visit_label = 'V01'

# Get all images from that candidate
r = json.loads(requests.get(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/dicoms',
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

prettyPrint(r)

In [ ]:
for file in r['Files']:
    filename = file['Filename']
    image = requests.get(
        url = baseurl + '/candidates/' + candid + '/' + visit_label + '/images/' + filename,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    )
    mincfile = open(filename, "w+b")
    mincfile.write(bytes(image.content))

## Exercise 8. Find all candidates and session with a given instruments

This is a series of GET request  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#31-specific-candidate  
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-candidate-instruments   
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

In [ ]:
instrument = 'test_all_fields'
request_count = 0

for candidate in candidates:
    candid = candidate['CandID']
    response = json.loads(requests.get(
        url = baseurl + '/candidates/' + candid,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))
    request_count += 1
    visit_labels = response['Visits']
    for visit_label in visit_labels:
        response = json.loads(requests.get(
            url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments',
            verify = False,
            headers = {'Authorization': 'Bearer %s' % token}
        ).content.decode('ascii'))
        request_count += 1
        if instrument in response['Instruments']:
            response = json.loads(requests.get(
                url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
                verify = False,
                headers = {'Authorization': 'Bearer %s' % token}
            ).content.decode('ascii'))
            request_count += 1
            print(json.dumps(response, indent=2, sort_keys=True))
            
print(request_count)